In [3]:
import os


In [4]:
! pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


In [6]:
# Read data
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
import splitfolders

input_folder = "/content/drive/My Drive/flowers/" 
output = "/content/drive/My Drive/Output_folder/" #Enter Output Folder

splitfolders.ratio(input_folder, output=output, seed=42, ratio=(0.7,0.2,0.1))

Copying files: 661 files [02:25,  2.68 files/s]

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/Output_folder/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/Output_folder/val',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
fianl_datagen = ImageDataGenerator(rescale = 1./255)
final_set = test_datagen.flow_from_directory('/content/drive/My Drive/Output_folder/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=5, activation='softmax'))

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = cnn.fit(x = training_set, validation_data = test_set, epochs = 15)

In [ ]:
loss, accuracy= cnn.evaluate(final_set)
print(f'Accuracy: {accuracy*100}')

In [ ]:
from tensorflow.keras.utils import plot_model
from PIL import Image

plot_model(cnn, to_file='cnn_model.png', show_shapes=True, show_layer_names=False, rankdir='TB', expand_nested=False, dpi=96)
display(Image.open('cnn_model.png'))

In [ ]:
import matplotlib.pyplot as plt

# Plot the loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
# Plot the accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
training_set.class_indices

In [ ]:
from io import BytesIO
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import img_to_array
import urllib

def loadImage(URL):
    with urllib.request.urlopen(URL) as url:
        img = load_img(BytesIO(url.read()), target_size=(64, 64))

    return img_to_array(img)

In [ ]:
import numpy as np
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
# test_image = load_img(img, target_size = (64, 64))
test_image = loadImage("https://media.istockphoto.com/id/1094016162/photo/a-field-of-pink-tulips-against-a-clear-cloudy-sky.jpg?s=2048x2048&w=is&k=20&c=6jh6xQhGgLXE_j_VqfPNJcEQNVc2pCrk5YU8hrwtZE0=")
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = cnn.predict(test_image)

In [ ]:
result

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

